In [1]:
import optimization as opt
import stock_work as sw
import pickle

In [2]:
sp500_tickers = sw.get_sp500_tickers()

In [3]:
try:
    with open("data.pkl", "rb") as f:
        data = pickle.load(f)

    stocks = data["stocks"]
    factors = data["factors"]

except:
    stocks, _ = sw.get_data(sp500_tickers)
    factors, _ = sw.get_factors()

with open("data.pkl", "wb") as f:
    pickle.dump({"stocks": stocks, "factors": factors}, f)

alphas, betas, idios, cov = sw.regression_results(stocks, factors)
target_betas = [.01, -.01, .01, -.01, .01, -.01, .01, -.01, .01, -.01, .01]

print(alphas.shape, betas.shape, idios.shape, cov.shape)

(16027, 503) (6717, 11)
(361, 500) (361, 12)
(500,) (11, 500) (500, 500) (11, 11)


In [4]:
betas.T

array([[ 0.09486977, -0.06397828,  0.13230069, ...,  0.2443274 ,
        -0.02009717,  0.16396406],
       [-0.36846516, -0.00908072,  0.2317802 , ..., -0.31298985,
        -0.0274149 , -0.33165661],
       [-0.13185477,  0.53414771,  0.53898687, ...,  0.42069252,
        -0.31455639, -0.10554992],
       ...,
       [-0.00537266, -0.2087839 ,  0.21318317, ...,  0.40642813,
         0.00891068, -0.14145623],
       [ 0.06698732,  0.24221405,  0.99736848, ...,  0.05873504,
         0.02161451, -0.03269267],
       [-0.07388795,  0.1621553 , -0.74902576, ...,  0.10458404,
         0.31735031,  0.33986501]])

In [5]:
vector_sum, betas, error, variance, iters = opt.optimize_constrainted_vector_sum(betas.T, target_betas, cov, idios, variance_bias = 0)

In [6]:
print("vector_sum:   ", vector_sum)
print("final_vector: ", betas)
print("error:        ", error)
print("variance:     ", variance)
print("iterations:   ", iters)

vector_sum:    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  2.10049405e-03  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.79840146e-03
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8.00546685e-03  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.000000